# Introduction

In my last post, I expressed a preference for working with channels as first class citizens.  Do you remember that flip-flop implementation?

```python
@coroutine
def flip_flop(outport):
    reg = 0
    while True:
        new = (yield)
        outport.send(reg)
        reg = new
```

This worked fine, but I would rather have a name for the thing that `new` is coming from.  Here's the type of function I'd like to work with.

In [4]:
def flip_flop(inport,outport):
    reg = 0
    while True:
        new = inport.take()
        outport.put(reg)
        reg = new


At a highlevel, I would want to test and interact with this flip-flop using code like this.

```python
inport = Channel()
outport = Channel()

go(flip_flop(inport,outport))

inport.put(1)
outport.take()
```

The `go` function is similar to what you would see in the Go, Clojure, and Rust programming languages.  For the time being, I'm going to focus on Go because of its simplicity.  Not only is the code simple to read, but concurrency is really one of it's selling points.

```go
package main

import (
	"fmt"
)

func flip_flop(port0 chan uint8, port1 chan uint8) {
	var state uint8 = 0
	for {
		in := <-port0
		port1 <- state
		state = in
	}
}

func main() {
	port0 := make(chan uint8)
	port1 := make(chan uint8)

	go flip_flop(port0, port1)

	inbits := []uint8{15, 10}

	var out_bit uint8
	var i uint8
	
	for _, x := range inbits {
		for i = 0; i < 8; i++ {
			in_bit := (x>> i) & 0x01
			port0 <- in_bit
			out_bit = <-port1
			fmt.Print(out_bit)
		}
	}
}

```